# Preparing Dos Dataset 

Import dataset

In [ ]:
import pandas as pd

df_thursday1 = pd.read_csv("/users/mik2002/aws-dataset/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv")
df_friday1 = pd.read_csv("/users/mik2002/aws-dataset/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv")


In [ ]:
df_dataset = pd.concat([df_thursday1,
                        df_friday1], axis=0, ignore_index=True)
df_dataset["Label"].value_counts()

In [ ]:
# Impostazione dell'opzione per mostrare tutte le colonne
pd.set_option('display.max_columns', None)

# Visualizzazione delle prime righe del DataFrame con tutte le colonne
df_dataset.head()

Dropping missing or duplicate values

In [ ]:
df_dataset# dropping missing or duplicate values
print("Before drop duplicate and missing values",df_dataset.shape)
df_dataset.drop_duplicates(inplace=True)
print("After drop duplicate values",df_dataset.shape)
df_dataset.dropna(inplace=True)
print("After drop missing values",df_dataset.shape)

# Creazione di una maschera booleana per identificare le righe con valore "Label" nella colonna "Label"
mask = df_dataset["Label"] == "Label"

# Rimozione delle righe identificate dalla maschera
df_dataset = df_dataset.drop(df_dataset[mask].index)


Replacing specific DoS attack labels with a general "Dos" category

In [ ]:
df_dataset.replace(to_replace=["DoS attacks-Hulk", "DoS attacks-SlowHTTPTest", "DoS attacks-GoldenEye","DoS attacks-Slowloris"], value="Dos", inplace=True)
df_dataset["Label"].value_counts()

Creating dummies based on the assumption that the Protocol values are categorical

In [ ]:
# Forcing them to be strings
df_dataset = df_dataset.astype({"Protocol": str})
df_dataset = pd.get_dummies(df_dataset, columns=['Protocol'], drop_first=True)
df_dataset["Protocol_17"] = df_dataset["Protocol_17"].astype(int)
df_dataset["Protocol_6"] = df_dataset["Protocol_6"].astype(int)
# making Label column the last column again
df_dataset.insert(len(df_dataset.columns)-1, 'Label', df_dataset.pop('Label'))

Dropping some columns that may be irrelevant (would require re-analysis)

In [ ]:
columns_to_drop = [
    'Dst Port',
    'Timestamp',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Fwd URG Flags',
    'Bwd URG Flags',
    'Flow Byts/s',  
    'Flow Pkts/s',
    'Fwd Byts/b Avg',
    'Fwd Pkts/b Avg',
    'Fwd Blk Rate Avg',
    'Bwd Byts/b Avg',
    'Bwd Pkts/b Avg',
    'CWE Flag Count',
    'Bwd Blk Rate Avg'  
]

In [ ]:
df_dataset.drop(columns=columns_to_drop, inplace=True)
df_dataset.shape

View the percentages of dataset

In [ ]:
count_labels = df_dataset["Label"].value_counts()
# Calculate the number of Benign and Malicious labels.
benign_count = count_labels.get("Benign", 0)
dos_count = count_labels.get("Dos", 0)

# Calculate percentages
total_count = benign_count + dos_count
benign_percent = (benign_count / total_count) * 100
dos_percent = (dos_count / total_count) * 100

# Print the percentages
print(f"Percentage of Benign labels: {benign_percent:.2f}%")
print(f"Percentage of Dos labels: {dos_percent:.2f}%")

Saving dataset

In [ ]:
df_dataset.info()

In [ ]:
df_dataset.to_csv("/users/mik2002/som/dataset-processed/processed_dos_dataset.csv", index=False)